# Co2 concentration in the atmosphere since 1958

# Introduction
In this notebook, I'll use the data of Mauna Loa Observatory, Hawaii provided by the [Scripps CO2 Program](https://scrippsco2.ucsd.edu/data/atmospheric_co2/primary_mlo_co2_record.html). The goal will be to explain the data using a model, and then try to predict the future concentration in atmosphere of co2 at this place until 2025.

In [199]:
import numpy as np
import pandas as pd
import os
import subprocess as sp
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
from calendar import month_abbr
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings("ignore")

## Load the data

Let's download the last version of the scripps program so that it's reusable for future work.
But the data we'll use in this notebook is a static dataset corresponding to the latest data available when the notebook was created for reproducibility

In [200]:
!curl https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/monthly/monthly_in_situ_co2_mlo.csv --output co2_data.csv
!cp co2_data.csv co2_data_new.csv
!cp ../input/scripps-co2-program-06122021/monthly_in_situ_co2_mlo.csv co2_data.csv

Let's check that the file has not been modified somehow by checking its hash

In [201]:
expected_hash = "645983b005053fb3ba442ad034966c7f23173975"
assert sp.getoutput("sha1sum ./co2_data.csv")[:len(expected_hash)] == expected_hash, "File has been somehow corrupted"

In [202]:
# Let's load the data !
data = pd.read_csv("./co2_data.csv", comment='"', skiprows=[55, 56])
# According to the website, an unavailable data point is represented by the value -99.99
data[data == -99.99] = np.NaN
data

Let's see if we can make some assumptions about the data and check them

In [203]:
# I expect that the date columns are not contradictory
if not (data.iloc[:, 0].values == np.floor(data.iloc[:, 3].values)).all(): warnings.warn("Date columns are contradictory !")
# I expect that if one value is missing, then every other one the same date are missing
if not ((data.values == -99.99).sum(axis=-1) == 6).all(): warnings.warn("There are entries where some values are missing but not all of them")

We'll consider only a subset of  the variables of the dataset, namely the measured co2 concentration, and the date expressed in a continuous way. The other columns are either redundancies or calculated columns.

In [204]:
# Separate the data that interest us
mois = data.iloc[:, 1]
annees = data.iloc[:, 3]
co2 = data.iloc[:, 4]

In [205]:
plt.title("C02 concentration (ppm) in athmosphere during time")
plt.plot(annees, co2, label="CO2 concentration")
plt.legend()
plt.xlabel("time")
plt.ylabel("C02 concentration (ppm)")
plt.show()

# Models
Now that the data is loaded and that we had a look at it, we are gonna fit models to it and try to express in the siplest way possible the data.

### Notation
During the notebook I'll describe the models using the following notation:
* $t$ denotes the date expressed in a continous way
* $y_t$ is the concentration of CO2 in the atmosphere at time $t$
* $\beta_i$ denotes the parameters of the model

## Linear model
As it's better to start simple, i'll first only try to explain the data using a linear model:
$$
    y_t = \beta_0 + \beta_1t
$$

In [206]:
"""
    Utils functions to re-use code
"""
def make_reg(X, y):
    """
        Make a regression using X to explain y
    """
    # Remove nans
    mask_y = np.logical_not(np.isnan(y))
    mask_x = np.logical_not(np.isnan(y))
    mask = np.logical_and(mask_x, mask_y)
    # Create a linear model with a bias
    reg = sm.OLS(y[mask].values, sm.add_constant(X[mask]))
    # Fit the model
    res = reg.fit()
    return res

def plot_predictions_vs_real(x, prediction, real):
    """
        Plot predictions against ground truth on the training data
    """
    plt.title("C02 concentration in athmosphere during time")
    plt.plot(x, real, label="Measured values")
    plt.plot(x, prediction, label="Predicted values")
    plt.legend()
    plt.xlabel("Years")
    plt.ylabel("CO2 concentration (ppm)")
    plt.show()

def plot_residuals(pred, y):
    """
        Plot residuals as a function of time and a lagged plot of residuals
    """
    residuals = y-pred
    plt.title("Residuals vs time")
    plt.plot(residuals, label="Residuals")
    plt.legend()
    plt.xlabel("Years")
    plt.ylabel("Residual")
    plt.show()
    plt.title("Residuals lagged scatter plot")
    plt.scatter(residuals[:-1], residuals[1:], label="Residuals")
    plt.legend()
    plt.xlabel("residuals at time t")
    plt.ylabel("residuals at time t + 1")
    plt.show()

In [207]:
# Now let's use our function to fit our first model
res = make_reg(annees, co2)
res.summary()

We see that the $R^2$ is very good ($0.976$). Furthermore, the p-values of the coefficient are very low which is a good sign.
Now let's visually see the fitte curve

In [208]:
co2_pred = res.predict(sm.add_constant(annees))
plot_predictions_vs_real(annees, co2_pred, co2)

We clearly see that the linear explains well the global trend. However it's clear that the true data is not linear. Maybe plotting the residuals could be helpfull

In [209]:
plot_residuals(co2_pred, co2)

So we clearly see that there is a structure with the shape of a banana in the residuals over time. This is not a good sign as some assumptions of the model are completly false. Furthermore, the impression is confirmed by the lagged plot

*Red flag: structure in the residuals*

## Exponential model
Visually, it looks like the curve is following a slite exponential-ish tendancy. Let's try to fit to $\log(y)$ instead of $y$ itself:
$$
    \log(y_t) = \beta_0 + \beta_1t
$$

In [210]:
res = make_reg(annees, np.log(co2))
res.summary()

The $R^2$ of $0.984$ is better than the previous one. The p-values indicate also a strong effect of the explainatory variables. I don't understand why the AIC is so low I thnk it's better to not take it into account for this model.

In [211]:
co2_pred = np.exp(res.predict(sm.add_constant(annees)))
plot_predictions_vs_real(annees, co2_pred, co2)

Visually, the log model looks better than the previous one, but we still see that it is not good enough. On the boundaries of the data, the error is pretty big, which is a problem if we want to extrapolate

In [212]:
plot_residuals(co2_pred, co2)

As in the linear model, we still have a banana structure in the residuals over time. Same for the lag plot .

*Red flag: still structure in the data*

## Polynomial model
Ok so the growth isn't linear or exponential, so I'll now assume that it is quadratic. To see if it's the case, let's fit a polynomial model:
$$
    y_t = \beta_0 + \beta_1t + \beta_2t^2
$$

In [213]:
res = make_reg(np.stack((annees, annees**2), axis=-1), co2)
res.summary()

Once again we have an even better $R^2$. All 3 p-values are very small meaning that all explainatory variable have a linear effect.
The AIC of $3373$ is the best one so far, which is a good indication that we are not yet overfitting

In [214]:
co2_pred = res.predict(sm.add_constant(np.stack((annees, annees**2), axis=-1)))
plot_predictions_vs_real(annees, co2_pred, co2)

The visual plot looks much better, we have visually the best possible explaintation for the global tendancy

In [215]:
plot_residuals(co2_pred, co2)

No clear structure other than the seasonal component in the time vs residuals plot. The mean and std appear to be the same during time. 
This seasonnal component can explain the correlation observed in the lagged plot. 
We should probably try to describe it

# Seasonal component
*Note of the me from the future: This section was quite a mess and end up on nothing usefull, i left it so that we can follow the path of toughts that led to the end but nothing usefull for the study until the next session*

## Fourier decomposition approach

In [216]:
periodic = co2_pred - co2
plt.title("Periodic component left")
plt.plot(annees, periodic)
plt.xlabel("years")
a = plt.ylabel("residuals")


I want to try to conduct a fourier transform decomposition, but there are some missing datapoints.
fft expect a uniformly sampled signal.
There is multiple things I can do from here:
1. Apply another harmonic decomposition algorithm that do not expect a uniformly sampled signal
2. Find an interpolation scheme to "fill in" the missing datapoints (that may introduce bias)
3. Perform my analysis on the biggest slice without missing datapoints

I've made some research to find a non uniform fourier transform algorithm but all I found was approximations, and algorithm where I have no clue how to use them. As I do not have a signal processing expert under the hand to ask questions to.<br>
[More details here](https://cims.nyu.edu/cmcl/nufft/nufft.html)

Now to choose between the second and last option I want to have a look at the location of the missing data points

In [217]:
na_mask = periodic.isna()
print("Date of missing values:", list(zip(np.floor(annees[na_mask]).astype(np.int).tolist(), mois[na_mask].tolist())))

I see that I have a slice between April 1964 and October 2021 with no missing values that could be used while dropping the rest. Another argument in favor of the slice option versus the interpolation one is that there are consectuive missing values, meaning that interpolation will be less precise.

So I've decided to go with the slice option.

In [218]:
print(np.arange(len(periodic))[na_mask])

In [219]:
periodic_slice = periodic[76:765]
assert not periodic_slice.isna().any(), "Bad indices, there is still nans in the data"
periodic_slice = periodic_slice.values
print("Proportion of the data used to perform fft: {:.4f}".format(len(periodic_slice) / len(periodic)))

In [220]:
def fourierExtrapolation(x, t, n_harm=4):
    n = x.size
    x_freqdom = np.fft.rfft(x)
    plt.plot(x_freqdom)
    plt.show()
    indexes = np.argsort(np.absolute(x_freqdom))
    indexes_keep = indexes[-(1+n_harm*2):]
    x_approx = np.zeros_like(x_freqdom)
    x_approx[indexes_keep] = x_freqdom[indexes_keep]
    return np.fft.irfft(x_approx, t.max()+1)[t]

approx = fourierExtrapolation(periodic_slice, np.arange(len(periodic_slice)))
plt.plot(periodic_slice)
plt.plot(approx)
plt.show()
plt.plot(periodic_slice - approx)
plt.show()

Well, the result isn't very good, we see that there is a clear boundary effect that will hurt if we try to extrapolate.

Given this result, I want to try to go in another direction to predict the seasonal component.
I will rather try to adjust the existing model than decomposing the problem

## Autoregressive approach
As the fourier gave nothing of interest, i will try to caracterize the seasonal component using an autoregressive model, meaning that i'll use the past values to predict the futur ones.
The advantage is that it can express much more than a linear model. The disadvantage is that it's harder to use and when extrapolating, the error blow up in time as it uses past predictions to predict futur ones.

In [221]:
plot = plot_acf(periodic[~na_mask])

The ACF plot show the correlation of $y_t$ and $y_{t-x}$. On this plot we clearly see a sinusoidal component. The wavelength would be around 12 units of time. As the samples are given on the 15 of each month, a period of 12 samples correspond to exactly a year, which is intuitively explainable.

## Polynomial + autoregressive model
Given what we just said, we will try to improve our best model (the polynomial one) by adding the value of y that we are trying to predict with a lag of $12$ time units:

$$
    y_t = \beta_0 + \beta_1y_{t-12} + \beta_2t + \beta_3t^2
$$

In [222]:
# Transform time and co2 series into polynomial and lagged variables
def make_ploynomial_and_seasonal(x, ts):
    return np.stack((ts[:-12], x[12:], x[12:]**2), axis=-1)

x = make_ploynomial_and_seasonal(annees, co2)
m = ~np.isnan(x).any(axis=-1)
res = make_reg(x[m], co2[12:][m])
res.summary()

The model is now much better ! We have a $R^2$ of $1$ meaning that we explained all the explainable variance. The AIC is way better than the preivous one, meaning that we can believe that we don't overfit. P-value wise, we lost the treshold of $0.001$ but we are still better than $0.01$.

In [223]:
co2_pred = res.predict(sm.add_constant(make_ploynomial_and_seasonal(annees, co2)))
plot_predictions_vs_real(annees[12:], co2_pred, co2[12:])

Visually the predictions looks very good. Some spikes can be seen sometimes that are slightly off the predictions but other than that nothing to really say.

In [224]:
plot_residuals(co2_pred, co2[12:])

The residuals still have some seasonal component that is not captured by the model, but there is no clear frequency associated to it.

## Linear + seasonal model
As we saw earlier the coefficient associated to the $t^2$ term is very small. Let's try to simplify the model by removing it:
$$
    y_t = \beta_0 + \beta_1y_{t-12} + \beta_2t
$$

In [225]:
def make_linear_and_seasonal(x, ts):
    return np.stack((ts[:-12], x[12:]), axis=-1)

x = make_linear_and_seasonal(annees, co2)
m = ~np.isnan(x).any(axis=-1)
res = make_reg(x[m], co2[12:][m])
res.summary()

the $R^2$ is the same, and the p-values are very small. However, the AIC is slithly higher, so according to this metric it's worth it to keep the quadratic component in t he equation.

In [226]:
co2_pred = res.predict(sm.add_constant(make_linear_and_seasonal(annees, co2)))
plot_predictions_vs_real(annees[12:], co2_pred, co2[12:])

Visually there is no big difference with the previous model

In [227]:
plot_residuals(co2_pred, co2[12:])

Really similar to the previous one

# Test on the existing data
I will now test the considered models trained on a subset of the data, and see their performances on the remaining data. I don't argue that it proves anything, but at best it could prove us wrong.
AS  the final goal is to predict until 2025 and we are currently at the end of 2021, we will try to predict 2019, 2020, 2021 using past data

In [228]:
first_test_year = 2019
train_mask = annees < first_test_year
# Split into train and test set
annees_train, co2_train = annees[train_mask], co2[train_mask]
annees_test, co2_test = annees[~train_mask], co2[~train_mask]

def predictions_plot(preds=None, ci=None, zoom=False):
    plt.plot(annees_train, co2_train, label="train set")
    plt.plot(annees_test, co2_test, label="true label")
    if preds is not None:
        plt.plot(annees_test, preds, label="Predicted")
        plt.fill_between(annees_test, ci[:, 0], ci[:, 1], alpha=.3)
        # Plot MSE to have a numerical metric
        print("MSE={:.4f}".format(np.mean((preds - co2_test)**2)))
    if zoom:
        plt.title("Predicted CO2 concentration in ppm (zoomed)")
        x_window = [2018, 2022]
        m = np.logical_and(annees_test >= x_window[0], annees_test <= x_window[1])
        if preds is not None:
            data = np.concatenate((co2_test[m].ravel(), preds[m].ravel(), ci[m].ravel()))
        else:
            data = co2_test[m]
        plt.xlim(x_window)
        plt.ylim([np.nanmin(data)*0.99, np.nanmax(data)*1.01])
    else:
        plt.title("Predicted CO2 concentration in ppm")
    plt.xlabel("Time")
    plt.ylabel("CO2 concentration (ppm)")
    plt.legend()
    plt.show()
        
predictions_plot(zoom=True)

## Linear model

In [229]:
res = make_reg(annees_train, co2_train)
x = sm.add_constant(annees_test)
co2_pred = res.predict(x)
ci = res.get_prediction(x).conf_int()
predictions_plot(co2_pred, ci)
predictions_plot(co2_pred, ci, zoom=True)

As expected, the linear model is not very good, the true value are really under-estimated. Furthermore, the confidence interval is wrong, probably because the assumptions of the model are violated

## Log model

In [230]:
res = make_reg(annees_train, np.log(co2_train))
x = sm.add_constant(annees_test)
co2_pred = np.exp(res.predict(x))
ci = np.exp(res.get_prediction(x).conf_int())
predictions_plot(co2_pred, ci)
predictions_plot(co2_pred, ci, zoom=True)

The exponential model is less wrong but still underestimates a lot the ground truth. Here again, the confidence interval are wrong probably for violated assumptions reasons (residuals not independant)

## Polynomial model

In [231]:
res = make_reg(np.stack((annees_train, annees_train**2), axis=-1), co2_train)
x = sm.add_constant(np.stack((annees_test, annees_test**2), axis=-1))
co2_pred = res.predict(x)
ci = res.get_prediction(x).conf_int()
predictions_plot(co2_pred, ci)
predictions_plot(co2_pred, ci, zoom=True)

The polynomial model is much better (MSE 10 times smaller than previous one) but as expected, the seasonal component is completly ignored.
Once again confidence interval fail probably for the same reasons

## Polynomial + lagged variable

In [232]:
# The autoregressive model needs to predict gradualy as it can be necessary to rely on past predictions
def autoregressive_extrapolation(reg, f, annees_test, co2_train, min_lag=12):
    n = len(annees_test)
    preds = co2_train.values
    cis = np.zeros((len(co2_train), 2))
    cis[:, 0] = co2_train.values
    cis[:, 1] = co2_train.values
    while len(annees_test):
        co2_batch, annees_batch = preds[-min_lag:], annees_test[:min_lag]
        n_pred = len(annees_batch)
        annees_test = annees_test[min_lag:]
        preds = np.concatenate((preds, np.zeros(n_pred)))
        cis = np.concatenate((
            cis,
            np.stack((np.zeros(n_pred), np.zeros(n_pred)), axis=-1)
        ), axis=0)
        annees_batch = np.concatenate((np.zeros(min_lag), annees_batch))
        x = f(
            annees_batch,
            preds[-min_lag-n_pred:]
        )
        co2_pred = res.predict(sm.add_constant(x))
        # Lower bound calculated by considering the best case scenario
        ci_low = res.get_prediction(
            sm.add_constant(f(annees_batch, cis[-min_lag-n_pred:, 0]))
        ).conf_int()[:, 0]
        # Higher bound calculated by considering the worst case scenario
        ci_high = res.get_prediction(
            sm.add_constant(f(annees_batch, cis[-min_lag-n_pred:, 1]))
        ).conf_int()[:, 1]
        
        preds[-n_pred:] = co2_pred
        cis[-n_pred:, 0] = ci_low
        cis[-n_pred:, 1] = ci_high
        
    return preds[-n:], cis[-n:]

In [233]:
x = make_ploynomial_and_seasonal(annees_train, co2_train)
m = ~np.isnan(x).any(axis=-1)
res = make_reg(x[m], co2_train[12:][m])

co2_pred, ci = autoregressive_extrapolation(res, make_ploynomial_and_seasonal, annees_test, co2_train)
predictions_plot(co2_pred, ci)
predictions_plot(co2_pred, ci, zoom=True)

Visually, we can see that the model is very close to ground truth. I'm surprised that the confidence intervals are wrong given that I consider the worst possible case at each step. I belive that it's again due to the violated assumptions of linear models but if anyone have an idea please share it in comments.

## Linear + lagged variable

In [234]:
x = make_linear_and_seasonal(annees_train, co2_train)
m = ~np.isnan(x).any(axis=-1)
res = make_reg(x[m], co2_train[12:][m])

co2_pred, ci = autoregressive_extrapolation(res, make_linear_and_seasonal, annees_test, co2_train)
predictions_plot(co2_pred, ci)
predictions_plot(co2_pred, ci, zoom=True)

Same idea with the linear + seasonal. Do note however that the MSE is better for the polynomial than the linear one. At least it doesn't invalidate what we said about the AIC

# Final prediction

The best AIC so far is the polynomial one, so we will use it to make our predictions 

In [235]:
x = make_ploynomial_and_seasonal(annees, co2)
m = ~np.isnan(x).any(axis=-1)
res = make_reg(x[m], co2[12:][m])

# Predict until 2025
annees_to_predict = [2022, 2023, 2024]
# Average over the measure date in the month (no big difference)
dates_prop = np.unique(annees - np.floor(annees)).reshape((-1, 2)).mean(axis=-1)
annees_pred = np.repeat(annees_to_predict, len(dates_prop)) + np.repeat(np.expand_dims(dates_prop, axis=0), 3, axis=0).ravel()
# Last months of current year are None so we have to also predict them
annees_pred = np.concatenate((annees[-3:], annees_pred))

co2_pred, ci = autoregressive_extrapolation(res, make_ploynomial_and_seasonal, annees_pred, co2[:-3])
plt.title("Predicted CO2 concentration in ppm")
plt.plot(annees, co2, label="train")
plt.plot(annees_pred, co2_pred, label="predicted")
plt.fill_between(annees_pred, ci[:, 0], ci[:, 1], color='orange', alpha=.3)
plt.xlabel("Time")
plt.ylabel("CO2 concentration (ppm)")
plt.legend()
plt.show()

plt.title("Predicted CO2 concentration in ppm (zoomed)")
plt.plot(annees, co2, label="train")
plt.plot(annees_pred, co2_pred, label="predicted")
plt.fill_between(annees_pred, ci[:, 0], ci[:, 1], color='orange', alpha=.3)
plt.xlim([annees_pred.min(), annees_pred.max()])
plt.ylim([ci.min(), ci.max()])
plt.xlabel("Time")
plt.ylabel("CO2 concentration (ppm)")
plt.legend()
plt.show()

In [236]:
new_data = pd.read_csv("./co2_data_new.csv", comment='"', skiprows=[55, 56])
new_annee = new_data.iloc[:, 3].values
new_co2 = new_data.iloc[:, 4].values
m = new_annee >= annees_pred.min()
new_annee = new_annee[m]
new_co2 = new_co2[m]
n_missing = len(annees_pred) - len(new_annee)
if n_missing > 0:
    new_annee = np.concatenate((new_annee, annees_pred[-n_missing:]))
    new_co2 = np.concatenate((new_co2, np.ones(n_missing) * -99.99))
new_co2[new_co2 == -99.99] = np.nan

In [237]:
extrapolation = pd.DataFrame({
    "Year": np.floor(annees_pred).astype(np.int),
    "Month": map(lambda x: month_abbr[int(x)+1], np.floor((annees_pred - np.floor(annees_pred))*12)),
    "Co2 Prediction": co2_pred,
    "Co2 True value": new_co2,
    "0.05 ci": ci[:, 0],
    "0.95 ci": ci[:, 1],
    "In Interval": np.logical_and(ci[:, 0] <= new_co2, new_co2 <= ci[:, 1]),
    "Squared error": (co2_pred - new_co2)**2
})
plt.title("Predicted CO2 concentration (in ppm) vs measured values")
plt.plot(annees, co2, label="train")
plt.plot(annees_pred, co2_pred, label="predicted")
plt.fill_between(annees_pred, ci[:, 0], ci[:, 1], color='orange', alpha=.3)
plt.plot(annees_pred, new_co2, label="measured")
plt.xlim([annees_pred.min(), annees_pred.max()])
plt.ylim([ci.min(), ci.max()])
plt.xlabel("Time")
plt.ylabel("CO2 concentration (ppm)")
plt.legend()
plt.show()
print("So far, MSE={:.4f}".format(np.nanmean(extrapolation["Squared error"])))
extrapolation

# Conclusion
We manage to fit a model using a quadratic growth + a seasonal (yearly) component with a test MSE less than $1$ppm. We failed to provide a valid confidence interval and we believe it may due to violated assumptions of linear models, namely the independance between error samples.
There is still periodic comopnents in the residuals that could be reolved by further work.